In [9]:
#Alicja's and Cristian's code. The aim is to scrape data from a Mobility News website and store it in a Data Frame. 
#After this, the next step would be to find an API to analyze the text data stored.

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.sharedmobility.news/category/mob/"
page_number = 1
article_count = 1

#store the data in lists
article_texts = []
article_urls = []
article_titles = []

#limit for the number of links to collect is set at 100. The code is very slow (or there is a lot of data)
# If we do not set the limit to 100 links, it takes too long to load and check if it properly works
link_limit = 100

while article_count <= link_limit:  #stop after 100 links
    #construct the URL for the current page
    url = f"{base_url}page/{page_number}/" if page_number > 1 else base_url

    response = requests.get(url)
    if response.status_code == 200:
        #parse the HTML
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        #links within the 'a' element within the main section
        link_elements = soup.find('main', class_='site-main rbc-content').find_all('a', class_='p-url')

        for link_element in link_elements:
            link = link_element.get('href')
            print("Link:", link)

            #access link and read its content
            article_response = requests.get(link)
            if article_response.status_code == 200:
                article_html_content = article_response.text
                article_soup = BeautifulSoup(article_html_content, 'html.parser')

                #get article title and text
                article_title = article_soup.find('h1', class_='entry-title').get_text()
                article_text = article_soup.find('div', class_='entry-content').get_text()

                #add data to lists
                article_titles.append(article_title)
                article_texts.append(article_text)
                article_urls.append(link)
                print("Article Title:", article_title)

                article_count += 1

        page_number += 1  #move to the next page
    else:
        print("Failed to retrieve the page. Status code:", response.status_code)
        break

#make a data frame
data = {
    "ID": range(1, article_count),
    "Text": article_texts,
    "URL": article_urls,
    "Article Title": article_titles
}

df = pd.DataFrame(data)
print(df.head())


Link: https://www.sharedmobility.news/the-role-of-microtransit-for-better-equity-and-inclusion-2/
Article Title: The role of microtransit for better equity and inclusion
Link: https://www.sharedmobility.news/now-centuries-old-last-mile-delivery-looks-to-a-profitable-future/
Article Title: Now centuries old, last-mile delivery looks to a profitable future
Link: https://www.sharedmobility.news/us-transit-leaders-praise-innovative-on-demand-programs/
Article Title: US: Transit leaders praise innovative on-demand programs
Link: https://www.sharedmobility.news/autonomous-vehicles-the-future-of-transportation/
Article Title: Autonomous vehicles: the future of transportation
Link: https://www.sharedmobility.news/new-shared-mobility-network-announces-visionary-2030-goals/
Article Title: New shared mobility network announces visionary 2030 goals
Link: https://www.sharedmobility.news/how-europes-cargo-bike-roll-out-is-delivering/
Article Title: How Europe’s cargo bike roll-out is delivering
Link